In [2]:
pip install cn2an

     |████████████████████████████████| 224 kB 700 kB/s            
     |████████████████████████████████| 137 kB 8.0 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

     |████████████████████████████████| 125 kB 719 kB/s            
     |████████████████████████████████| 40 kB 3.7 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import geopy.distance
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
import re
import cn2an
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge,ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from datetime import datetime
from geopy.distance import geodesic


In [5]:
# 读取数据
train_data = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_train.csv')
test_data = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_test.csv')
detail_data = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_details.csv')
rent_data = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_rent.csv')

In [6]:
# 删除多个列
train_data = train_data.drop(columns=['小区名称','抵押信息','房屋优势','套内面积',
                                        '户型介绍', '核心卖点'])
test_data = test_data.drop(columns=['小区名称','抵押信息','房屋优势','套内面积',
                                        '户型介绍', '核心卖点'])

In [7]:
# 处理异常值：删除房价异常的样本（使用 1.5 IQR 方法）
Q1 = train_data["价格"].quantile(0.25)
Q3 = train_data["价格"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
train_data = train_data[(train_data["价格"] >= lower_bound) & (train_data["价格"] <= upper_bound)]

# 统计去除异常值后的样本数量
num_predictions = train_data.shape[0]

In [8]:
#经纬度

# 定义目标城市的中心点经纬度
city_centers = {
    '北京': (39.9042,116.4074,),      # 天安门
    '燕郊': (39.9395,116.8036),     # 燕郊镇政府
    '武汉': (30.5928,114.3052),     # 江汉关
    '哈尔滨': (45.8022,126.5340),   # 中央大街
    '西安': (34.3416,108.9398),     # 钟楼
    '重庆': (29.5637,106.5507),     # 解放碑
    '上海': (31.2304,121.4737)      # 人民广场
}

In [9]:
def assign_city_and_distance(row):
    """
    根据经纬度判断所属城市，并计算到市中心的距离
    返回值：(城市名称, 距离_公里)
    """
    current_point = (row['lat'], row['lon'])  
    
    min_distance = float('inf')
    nearest_city = None
    
    for city, center in city_centers.items():
        try:
            distance = geodesic(current_point, center).km
        except:
            return (None, None)  # 处理无效坐标
        
        if distance < min_distance:
            min_distance = distance
            nearest_city = city
    
    return (nearest_city, min_distance)

# 应用函数到每一行
train_data[['预测城市', '距市中心距离_公里']] = train_data.apply(
    lambda row: pd.Series(assign_city_and_distance(row)), 
    axis=1
)
test_data[['预测城市', '距市中心距离_公里']] = test_data.apply(
    lambda row: pd.Series(assign_city_and_distance(row)), 
    axis=1
)
# 删除多个列
train_data = train_data.drop(columns=['预测城市'])
test_data = test_data.drop(columns=['预测城市'])

In [10]:
# 城市
train_data = pd.get_dummies(train_data, columns=['城市'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['城市'], drop_first=True)

In [11]:
# 周边配套

# 处理缺失值
train_data["周边配套"] = train_data["周边配套"].fillna("")
test_data["周边配套"] = test_data["周边配套"].fillna("")
# 规范化文本：去除特殊字符，统一分隔符
train_data["周边配套"] = train_data["周边配套"].apply(lambda x: re.sub(r"[，、。；：]", ",", x))  # 统一用逗号分隔
train_data["周边配套"] = train_data["周边配套"].apply(lambda x: re.sub(r"\s+", "", x))  # 去除多余空格
test_data["周边配套"] = test_data["周边配套"].apply(lambda x: re.sub(r"[，、。；：]", ",", x))  # 统一用逗号分隔
test_data["周边配套"] = test_data["周边配套"].apply(lambda x: re.sub(r"\s+", "", x))  # 去除多余空格


# 设定关键词类别
categories = ["医院", "公园", "超市", "商场", "银行", "学校", "地铁", "公交"]

# 创建新特征列，每个类别对应一个二值变量（是否出现）
for cat in categories:
    train_data[cat] = train_data["周边配套"].apply(lambda x: 1 if cat in x else 0)

for cat in categories:
    test_data[cat] = test_data["周边配套"].apply(lambda x: 1 if cat in x else 0)


In [12]:
#交通出行
# 处理缺失值
train_data["交通出行"] = train_data["交通出行"].fillna("")
test_data["交通出行"] = test_data["交通出行"].fillna("")
# 规范化文本：去除特殊字符，统一分隔符
train_data["交通出行"] = train_data["交通出行"].apply(lambda x: re.sub(r"[，、。；：]", ",", x))  # 统一用逗号分隔
train_data["交通出行"] = train_data["交通出行"].apply(lambda x: re.sub(r"\s+", "", x))  # 去除多余空格
test_data["交通出行"] = test_data["交通出行"].apply(lambda x: re.sub(r"[，、。；：]", ",", x))  # 统一用逗号分隔
test_data["交通出行"] = test_data["交通出行"].apply(lambda x: re.sub(r"\s+", "", x))  # 去除多余空格


# 设定关键词类别
categories = ["公交", "地铁", "高铁", "高速", "机场","轻轨"]

# 创建新特征列，每个类别对应一个二值变量（是否出现）
for cat in categories:
    train_data[cat] = train_data["交通出行"].apply(lambda x: 1 if cat in x else 0)

for cat in categories:
    test_data[cat] = test_data["交通出行"].apply(lambda x: 1 if cat in x else 0)


In [13]:
#  '环线'
train_data['环线'] = train_data['环线'].replace({
    '一环内': 1, '二环内': 2, '内环内': 0, '外环外': 6, '四环外': 5,
    '三环外': 4,'六环外': 7,'一至二环':1.5, '二至三环':2.5, '三至四环':3.5,
    '四至五环':4.5,'五至六环':5.5, '内环至外环':4, '内环至中环':3, '中环至外环':5
})

test_data['环线'] = test_data['环线'].replace({
    '一环内': 1, '二环内': 2, '内环内': 0, '外环外': 6, '四环外': 5,
    '三环外': 4,'六环外': 7,'一至二环':1.5, '二至三环':2.5, '三至四环':3.5,
    '四至五环':4.5,'五至六环':5.5, '内环至外环':4, '内环至中环':3, '中环至外环':5
})

In [14]:
# 02 房屋朝向
train_data["房屋朝向"] = train_data["房屋朝向"].apply(lambda x: 1 if "南" in str(x) else 0)

print(train_data[['房屋朝向']].head())

test_data["房屋朝向"] = test_data["房屋朝向"].apply(lambda x: 1 if "南" in str(x) else 0)


   房屋朝向
1     1
3     0
4     1
5     1
8     1


In [15]:
# 03 房屋户型（分为室、厅、厨、卫四列）
train_data[["室", "厅", "厨", "卫"]] = train_data["房屋户型"].str.extract(r'(\d+)室?(\d+)厅?(\d+)厨?(\d+)卫?').fillna(0).astype(int)

test_data[["室", "厅", "厨", "卫"]] = test_data["房屋户型"].str.extract(r'(\d+)室?(\d+)厅?(\d+)厨?(\d+)卫?').fillna(0).astype(int)

# 删除列
train_data = train_data.drop(columns=['房屋户型'])
test_data = test_data.drop(columns=['房屋户型'])

In [16]:
# 04 所在楼层
def process_floor_data(df):
    """
    处理楼层信息：
    1. 提取总楼层数(共X层)
    2. 直接对楼层位置（高/中/低/顶层/地下室）进行赋分
    """
    df = df.copy()
    
    # 1. 提取总楼层数
    df['总楼层数'] = df['所在楼层'].str.extract(r'共(\d+)层').astype(float)
    
    # 2. 直接计算楼层位置分数（不生成类型列）
    def get_floor_score(desc):
        desc = str(desc)
        if '地下室' in desc:
            return 0   # 地下室赋分0
        elif '顶层' in desc:
            return 3   # 顶层赋分3
        elif '高楼层' in desc:
            return 4   # 高楼层赋分4
        elif '中楼层' in desc:
            return 5   # 中楼层赋分5（通常最受欢迎）
        elif '低楼层' in desc or '底层' in desc:
            return 2   # 低楼层赋分2
        else:
            return 1   # 其他情况赋分1
    
    df['楼层位置分数'] = df['所在楼层'].apply(get_floor_score)
    
    # 3. 填充缺失的总楼层数（用同类型分数的平均楼层数）
    if df['总楼层数'].isna().any():
        floor_avg = df.groupby('楼层位置分数')['总楼层数'].mean()
        for score in floor_avg.index:
            mask = (df['楼层位置分数'] == score) & (df['总楼层数'].isna())
            df.loc[mask, '总楼层数'] = floor_avg[score]
    
    # 4. 删除原始列
    if '所在楼层' in df.columns:
        df = df.drop(columns=['所在楼层'])
    
    return df

# 处理训练集和测试集
train_data = process_floor_data(train_data)
test_data = process_floor_data(test_data)

# 验证结果
print("训练集楼层特征：")
print(train_data[['总楼层数', '楼层位置分数']].describe())
print("\n测试集楼层特征:")
print(test_data[['总楼层数', '楼层位置分数']].describe())

训练集楼层特征：
               总楼层数        楼层位置分数
count  75927.000000  75927.000000
mean      19.630263      3.708035
std       11.299866      1.276252
min        0.000000      0.000000
25%        8.000000      2.000000
50%       18.000000      4.000000
75%       31.000000      5.000000
max       74.000000      5.000000

测试集楼层特征:
               总楼层数        楼层位置分数
count  14786.000000  14786.000000
mean      16.901393      3.801975
std       10.798503      1.240239
min        1.000000      0.000000
25%        6.000000      2.000000
50%       15.000000      4.000000
75%       27.000000      5.000000
max       58.000000      5.000000


In [17]:
# 05 建筑面积

# 删除 '建筑面积' 列中的 '㎡' 单位，并转换为浮动数值
train_data['建筑面积'] = train_data['建筑面积'].str.replace('㎡', '').astype(float)
test_data['建筑面积'] = test_data['建筑面积'].str.replace('㎡', '').astype(float)

# 查看处理后的数据
print(train_data[['建筑面积']].head())



     建筑面积
1  127.44
3   43.60
4   39.85
5   69.30
8   90.60


In [18]:
# 06 建筑结构
# 新版建筑结构评分（0-10分制）
structure_scores = {
    '钢混结构': 10,  # 最优
    '钢结构': 8,
    '框架结构': 7,
    '混合结构': 5,
    '砖混结构': 3,
    '砖木结构': 1,   # 最差
    '未知结构': 2    # 保守处理
}

# 训练集转换
train_data['建筑结构分数'] = train_data['建筑结构'].map(structure_scores)
# 测试集转换（使用相同的映射）
test_data['建筑结构分数'] = test_data['建筑结构'].map(structure_scores)

# 检查转换结果
print("训练集建筑结构分布：")
print(train_data['建筑结构分数'].value_counts().sort_index())
print("\n测试集建筑结构分布:")
print(test_data['建筑结构分数'].value_counts().sort_index())

# 删除原始列
train_data = train_data.drop(columns=['建筑结构'])
test_data = test_data.drop(columns=['建筑结构'])


训练集建筑结构分布：
1.0        54
2.0      1671
3.0      3493
5.0      7776
7.0      2445
8.0       963
10.0    58920
Name: 建筑结构分数, dtype: int64

测试集建筑结构分布:
1.0         7
2.0       388
3.0      1330
5.0      1875
7.0       310
8.0        80
10.0    10794
Name: 建筑结构分数, dtype: int64


In [19]:
# 07 装修情况
#定义装修情况与对应分数的映射关系（根据优劣来赋分）
orientation_scores = {
    '精装': 9,
    '简装': 6,
    '其他': 1,
    '毛坯': 4,
}

# 将'建筑结构'列中的朝向替换为对应的分数
train_data['装修情况'] = train_data['装修情况'].replace(orientation_scores)
test_data['装修情况'] = test_data['装修情况'].replace(orientation_scores)
# 查看处理后的数据
print(train_data[['装修情况']].head())

   装修情况
1   9.0
3   9.0
4   9.0
5   9.0
8   9.0


In [20]:
# 08 梯户比例

def convert_ratio_to_float(text):
    try:
        # 处理空值和字符串格式化
        if pd.isna(text) or str(text).strip() in ['', 'nan', 'None']:
            return None
        
        text = str(text).strip()
        
        # 更宽松的正则匹配（允许空格和其他分隔符）
        match = re.search(r"([一二三四五六七八九十两\d]+)\s*梯\s*([一二三四五六七八九十两\d]+)\s*户", text)
        if not match:
            # 尝试匹配阿拉伯数字格式（如2梯4户）
            match = re.search(r"(\d+)\s*梯\s*(\d+)\s*户", text)
        
        if match:
            梯 = match.group(1)
            户 = match.group(2)
            
            # 统一转换为数字
            梯_num = float(cn2an.cn2an(梯, "smart") if 梯.isalpha() else 梯)
            户_num = float(cn2an.cn2an(户, "smart") if 户.isalpha() else 户)
            
            return 梯_num / 户_num
        return None
    except Exception as e:
        print(f"转换失败: {text}, 错误: {str(e)}")
        return None

# 应用转换
train_data["梯户比例"] = train_data["梯户比例"].apply(convert_ratio_to_float)
test_data["梯户比例"] = test_data["梯户比例"].apply(convert_ratio_to_float)

# 填充缺失值
train_data["梯户比例"] = train_data["梯户比例"].fillna(train_data["梯户比例"].median())
test_data["梯户比例"] = test_data["梯户比例"].fillna(train_data["梯户比例"].median())  # 使用训练集的统计量

print("\n训练集结果:")
print(train_data["梯户比例"].value_counts(dropna=False).head())


训练集结果:
0.500000    25687
0.333333    12660
0.250000     8596
0.375000     4612
1.000000     3010
Name: 梯户比例, dtype: int64


In [21]:
# 09 '配备电梯'
train_data['配备电梯'] = train_data['配备电梯'].replace({
    '有': 1,
    '无': 0,
   
})
print(train_data[['配备电梯']].head())
test_data['配备电梯'] = test_data['配备电梯'].replace({
    '有': 1,
    '无': 0,
   
})

   配备电梯
1   0.0
3   1.0
4   1.0
5   1.0
8   0.0


In [22]:
# 10 交易权属
# 交易权属评分映射 (0-100)
score_map = {
    "商品房": 100,
    "私产": 85,
    "自住型商品房": 80,
    "央产房": 78,
    "已购公房": 70,
    "集资房": 65,
    "房改房": 60,
    "定向安置房": 55,
    "限价商品房": 45,
    "经济适用房": 40,
    "一类经济适用房": 38,
    "二类经济适用房": 35,
    "动迁安置房": 30,
    "拆迁还建房": 28,
    "使用权": 20,
    "售后公房": 15
}

# 创建新列
train_data["交易权属评分"] = train_data["交易权属"].map(score_map)
test_data["交易权属评分"] = test_data["交易权属"].map(score_map)
# 删除列
train_data = train_data.drop(columns=['交易权属'])
test_data = test_data.drop(columns=['交易权属'])

In [23]:
#13 房屋用途

用途赋分 = {
    "普通住宅": 90, "别墅": 95, "花园洋房": 95, "四合院": 98, "新式里弄": 92, 
    "公寓": 85, "老公寓": 80, "住宅式公寓": 80, "公寓（住宅）": 80, "公寓/住宅": 80,  
    "商住两用": 70, "酒店式公寓": 65, "商业办公类": 60, "写字楼": 50, "商业": 40, "底商": 50, 
    "车库": 30, "公寓/公寓": 85, "平房": 75
}

train_data["房屋用途分数"] = train_data["房屋用途"].map(用途赋分)
test_data["房屋用途分数"] = test_data["房屋用途"].map(用途赋分)

train_data = train_data.drop(columns=['房屋用途'])
test_data = test_data.drop(columns=['房屋用途'])


In [24]:
#14 房屋年限

年限赋分 = {
    "满五年": 100,
    "满两年": 80,
    "未满两年": 60
}
train_data["房屋年限分数"] = train_data["房屋年限"].map(年限赋分)
test_data["房屋年限分数"] = test_data["房屋年限"].map(用途赋分)

train_data = train_data.drop(columns=['房屋年限'])
test_data = test_data.drop(columns=['房屋年限'])


In [25]:
#15 产权所属

产权赋分 = {
    "非共有": 10,
    "共有": 6  # 共有因交易复杂性，赋较低分
}
train_data["产权分数"] = train_data["产权所属"].map(产权赋分)
test_data["产权分数"] = test_data["产权所属"].map(产权赋分)

train_data = train_data.drop(columns=['产权所属'])
test_data = test_data.drop(columns=['产权所属'])


In [26]:
# 16 别墅类型（数值越大，市场价值越高）
villa_value = {
    '独栋': 4,  # 最高端
    '双拼': 3,
    '叠拼': 2, 
    '联排': 1,  # 最低端
}

# 标准化到 0-10 分范围
max_value = max(villa_value.values())  # 最大值（4）
scaled_villa_value = {k: (v / max_value) * 10 for k, v in villa_value.items()}

# 修改原列：无信息的返回 0，其他按市场价值赋分
train_data['别墅类型'] = train_data['别墅类型'].apply(
    lambda x: 0 if pd.isna(x) or str(x).strip() == '' else scaled_villa_value.get(x, 0)
)
test_data['别墅类型'] = test_data['别墅类型'].apply(
    lambda x: 0 if pd.isna(x) or str(x).strip() == '' else scaled_villa_value.get(x, 0)
)
# 检查修改后的数据
print(train_data['别墅类型'].value_counts())

0.0     75410
2.5       336
5.0       148
7.5        27
10.0        6
Name: 别墅类型, dtype: int64


In [27]:
#交易时间

def process_transaction_data(train_df, test_df):
    """
    统一处理训练集和测试集的交易时间特征：
    1. 日期标准化
    2. 特征工程
    3. 使用训练集统计量填充测试集
    """
    # 复制数据避免污染原始数据
    train = train_df.copy()
    test = test_df.copy()
    
    # === 1. 日期格式标准化 ===
    for df in [train, test]:
        df['上次交易'] = pd.to_datetime(df['上次交易'], errors='coerce', format='%Y-%m-%d')
        df['交易时间'] = pd.to_datetime(df['交易时间'], errors='coerce', format='%Y-%m-%d')
    
    # === 2. 提取基础特征 ===
    for df in [train, test]:
        df['上次年份'] = df['上次交易'].dt.year
        df['交易年份'] = df['交易时间'].dt.year
        df['持有天数'] = (df['交易时间'] - df['上次交易']).dt.days
        df['持有年限'] = df['持有天数'] / 365.25  # 精确计算
    
    # === 3. 交易频率计算（仅基于训练集统计）===
    train_transaction_counts = train['交易时间'].value_counts().to_dict()
    train['交易频率'] = train['交易时间'].map(train_transaction_counts)
    test['交易频率'] = test['交易时间'].map(train_transaction_counts)  # 使用训练集统计量
    
    # === 4. 异常值处理 ===
    current_year = pd.Timestamp.now().year
    
    # 统一异常值范围（基于训练集统计）
    min_hold_days = train['持有天数'].quantile(0.01)  # 取训练集1%分位数
    max_hold_days = train['持有天数'].quantile(0.99)  # 取训练集99%分位数
    
    for df in [train, test]:
        # 处理未来日期
        df.loc[df['上次年份'] > current_year, '上次年份'] = np.nan
        df.loc[df['交易年份'] > current_year, '交易年份'] = np.nan
        
        # 处理不合理持有时间
        df.loc[df['持有天数'] < min_hold_days, '持有天数'] = np.nan
        df.loc[df['持有天数'] > max_hold_days, '持有天数'] = np.nan
        df['持有年限'] = df['持有天数'] / 365.25  # 重新计算
    
    # === 5. 填充缺失值（使用训练集统计量）===
    # 计算训练集填充值
    fill_values = {
        '上次年份': train['上次年份'].median(),
        '持有年限': train['持有年限'].median(),
        '交易频率': 1  # 默认最低频率
    }
    
    # 统一填充
    for col in fill_values:
        train[col] = train[col].fillna(fill_values[col])
        test[col] = test[col].fillna(fill_values[col])  # 使用相同填充值
    
    # === 6. 生成衍生特征 ===
    for df in [train, test]:
        # 是否快速交易（2年内）
        df['是否快速交易'] = (df['持有年限'] <= 2).astype(int)
        
        # 交易季节（1-4季度）
        df['交易季度'] = df['交易时间'].dt.quarter
        
        # 是否旺季（假设Q2/Q3为旺季）
        df['是否旺季'] = df['交易季度'].isin([2, 3]).astype(int)
    
    # === 7. 清理中间列 ===
    cols_to_drop = ['持有天数']  # 保留更直观的"持有年限"
    train = train.drop(columns=cols_to_drop, errors='ignore')
    test = test.drop(columns=cols_to_drop, errors='ignore')
    
    return train, test

# 应用处理
train_data, test_data = process_transaction_data(train_data, test_data)

# 验证结果
print("=== 训练集特征样例 ===")
print(train_data[['上次交易', '交易时间', '持有年限', '交易频率', '是否快速交易']].head())
print("\n=== 测试集特征样例 ===")
print(test_data[['上次交易', '交易时间', '持有年限', '交易频率', '是否快速交易']].head())

print("\n=== 训练集统计 ===")
print(f"持有年限中位数: {train_data['持有年限'].median():.1f}年")
print(f"快速交易比例: {train_data['是否快速交易'].mean():.1%}")

print("\n=== 测试集统计 ===")
print(f"持有年限中位数: {test_data['持有年限'].median():.1f}年")
print(f"快速交易比例: {test_data['是否快速交易'].mean():.1%}")

=== 训练集特征样例 ===
        上次交易       交易时间      持有年限  交易频率  是否快速交易
1 2010-12-10 2020-03-13  9.256674     2       0
3 2016-09-05 2019-02-14  2.442163     1       0
4 2017-02-28 2019-04-26  2.154689     1       0
5 2016-10-18 2020-07-15  3.739904     1       0
8 2017-05-22 2020-03-23  2.836413     1       0

=== 测试集特征样例 ===
        上次交易       交易时间       持有年限  交易频率  是否快速交易
0 2019-06-17 2024-08-18   5.171800   1.0       0
1 2003-09-20 2024-10-27  21.103354   1.0       0
2 2006-12-30 2024-12-09  17.943874   1.0       0
3 2004-06-22 2024-07-31  20.106776   1.0       0
4 2013-05-10 2024-07-31  11.225188   1.0       0

=== 训练集统计 ===
持有年限中位数: 5.9年
快速交易比例: 6.5%

=== 测试集统计 ===
持有年限中位数: 6.5年
快速交易比例: 6.2%


In [28]:
# 删除多个列
train_data = train_data.drop(columns=['交易时间','上次交易'])
test_data = test_data.drop(columns=['交易时间','上次交易'])

In [29]:
# 获取所有不同的类型及其出现次数
house_type_counts = train_data['梯户比例'].value_counts()

# 查看结果
print(house_type_counts)

0.500000     25687
0.333333     12660
0.250000      8596
0.375000      4612
1.000000      3010
             ...  
20.000000        1
0.085714         1
0.049180         1
0.021277         1
0.040816         1
Name: 梯户比例, Length: 165, dtype: int64


In [30]:
# 缺失值处理

fill_rules = {
    '环线': train_data['环线'].quantile(0.75),  # 使用75分位数填充
    '建筑结构分数': train_data['建筑结构分数'].mean(),  # 使用均值填充
    '装修情况': train_data['装修情况'].mean(),
    '配备电梯': train_data['配备电梯'].mean(),
    '房屋年限分数': train_data['房屋年限分数'].mean(),
    '房屋用途分数': train_data['房屋用途分数'].mean()
}

# 打印填充前的缺失值情况
print("【填充前】训练集缺失值统计:")
print(train_data[fill_rules.keys()].isnull().sum())

print("\n【填充前】测试集缺失值统计:")
print(test_data[fill_rules.keys()].isnull().sum())

# 填充训练集
print("\n=== 开始填充 ===")
for col, fill_value in fill_rules.items():
    print(f"正在填充 {col}: 使用值 {fill_value:.2f}")
    train_data[col] = train_data[col].fillna(fill_value)
    test_data[col] = test_data[col].fillna(fill_value)  # 使用相同的值填充测试集

# 验证填充结果
print("\n【填充后】训练集缺失值统计:")
print(train_data[fill_rules.keys()].isnull().sum())

print("\n【填充后】测试集缺失值统计:")
print(test_data[fill_rules.keys()].isnull().sum())

# 检查填充值是否一致
print("\n=== 填充值验证 ===")
for col in fill_rules.keys():
    train_non_null = train_data[col].notnull().all()
    test_non_null = test_data[col].notnull().all()
    print(f"{col}: 训练集是否全部填充: {train_non_null}, 测试集是否全部填充: {test_non_null}")

【填充前】训练集缺失值统计:
环线        40860
建筑结构分数      605
装修情况        605
配备电梯       7830
房屋年限分数    29498
房屋用途分数        2
dtype: int64

【填充前】测试集缺失值统计:
环线         5458
建筑结构分数        2
装修情况          2
配备电梯       1298
房屋年限分数    14786
房屋用途分数        0
dtype: int64

=== 开始填充 ===
正在填充 环线: 使用值 4.50
正在填充 建筑结构分数: 使用值 8.85
正在填充 装修情况: 使用值 5.15
正在填充 配备电梯: 使用值 0.78
正在填充 房屋年限分数: 使用值 87.01
正在填充 房屋用途分数: 使用值 88.76

【填充后】训练集缺失值统计:
环线        0
建筑结构分数    0
装修情况      0
配备电梯      0
房屋年限分数    0
房屋用途分数    0
dtype: int64

【填充后】测试集缺失值统计:
环线        0
建筑结构分数    0
装修情况      0
配备电梯      0
房屋年限分数    0
房屋用途分数    0
dtype: int64

=== 填充值验证 ===
环线: 训练集是否全部填充: True, 测试集是否全部填充: True
建筑结构分数: 训练集是否全部填充: True, 测试集是否全部填充: True
装修情况: 训练集是否全部填充: True, 测试集是否全部填充: True
配备电梯: 训练集是否全部填充: True, 测试集是否全部填充: True
房屋年限分数: 训练集是否全部填充: True, 测试集是否全部填充: True
房屋用途分数: 训练集是否全部填充: True, 测试集是否全部填充: True


In [31]:
# 价格分布
sns.histplot(train_data["价格"], bins=50, kde=True)
plt.title("房价分布")
plt.show()

<Figure size 432x288 with 1 Axes>

**建模**

In [44]:
# 选择数值变量进行建模
features = [
    "区域","板块","建筑面积","房屋朝向","装修情况","梯户比例","配备电梯", "别墅类型","房屋年限分数", 
    "产权分数", "房屋用途分数", "环线", "年份","楼层位置分数","建筑结构分数",
    "室", "厅", "厨", "卫", "总楼层数", "交易权属评分",  "lon", "lat", 
    "持有年限", "交易频率", "是否快速交易",
    "医院", "公园", "超市", "商场", "银行", "学校", "地铁", "公交",
     "高铁", "高速", "机场","轻轨","城市_1" ,"城市_2" , "城市_3",  "城市_4", "城市_5",  "城市_6",
     "距市中心距离_公里"
]

X = train_data[features]

#交互项
X["房屋年限 * 产权分数"] = X["房屋年限分数"] * X["产权分数"]

y = np.log1p(train_data["价格"])  # 使用对数价格建模


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [45]:
# 数据拆分 (80% 训练集, 20% 测试集)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [46]:
# 模型评估函数
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    """
    综合评估模型性能
    
    参数:
        model: 已训练的模型对象
        X_train: 训练集特征
        y_train: 训练集目标值
        X_test: 测试集特征
        y_test: 测试集目标值
        model_name: 模型名称标识
        
    返回:
        包含各项评估指标的字典
    """
    # 样本内评估
    y_train_pred = model.predict(X_train)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mid = median_absolute_error(y_train, y_train_pred)

    # 样本外评估
    y_test_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mid = median_absolute_error(y_test, y_test_pred)

    # 交叉验证评估
    kf = KFold(n_splits=6, shuffle=True, random_state=42)
    
    # 使用cross_validate获取多个评分
    cv_results = cross_validate(
        model, 
        X_train, 
        y_train,
        cv=kf,
        scoring={
            'mae': 'neg_mean_absolute_error',
            'mse': 'neg_mean_squared_error'
        },
        return_train_score=False
    )
    
    cv_mae = -np.mean(cv_results['test_mae'])
    cv_rmse = np.sqrt(-np.mean(cv_results['test_mse']))

    return {
        'Model': model_name,
        'In-sample MAE': train_mae,
        'In-sample RMSE': train_rmse,
        'In-sample Median AE': train_mid,
        'Out-of-sample MAE': test_mae,
        'Out-of-sample RMSE': test_rmse,
        'Out-of-sample Median AE': test_mid,
        'Cross-validation MAE': cv_mae,
        'Cross-validation RMSE': cv_rmse
    }
# 初始化不同回归模型
models = {
    'OLS': LinearRegression(),  # 普通最小二乘
    'LASSO': Lasso(alpha=1, tol=1e-4, max_iter=10000),  # L1正则化
    'Ridge': Ridge(alpha=1.0),  # L2正则化
    'ElasticNet': ElasticNet(alpha=1, l1_ratio=0.5, max_iter=10000)  # 弹性网络
}
# 使用示例
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    results.append(evaluate_model(model, X_train, y_train, X_test, y_test, name))

results_df = pd.DataFrame(results)


In [47]:
print("DataFrame 实际包含的列:", results_df.columns.tolist())

DataFrame 实际包含的列: ['Model', 'In-sample MAE', 'In-sample RMSE', 'In-sample Median AE', 'Out-of-sample MAE', 'Out-of-sample RMSE', 'Out-of-sample Median AE', 'Cross-validation MAE', 'Cross-validation RMSE']


In [48]:
# 模型训练与评估流程
results = []
for name, model in models.items():
    print(f"正在训练 {name} 模型...")
    model.fit(X_train, y_train)
    results.append(evaluate_model(model, X_train, y_train, X_test, y_test, name))
    print(f"{name} 模型训练完成")

# 转换为DataFrame并格式化输出
results_df = pd.DataFrame(results)
results_df = results_df.round(4)  # 保留4位小数


# 选择最佳模型（基于测试集MAE）
best_model_info = results_df.loc[results_df['Out-of-sample MAE'].idxmin()]  # 使用正确的列名
best_model_name = best_model_info['Model']
best_model = models[best_model_name]

# 打印评估结果
print("\n" + "="*50)
print("各模型性能比较（数值越小越好）:")

# 根据实际列名定义显示顺序
column_order = [
    'Model',
    'In-sample MAE', 
    'Out-of-sample MAE',
    'Cross-validation MAE',
    'In-sample RMSE',
    'Out-of-sample RMSE', 
    'Cross-validation RMSE',
    'In-sample Median AE',
    'Out-of-sample Median AE'
]

# 打印完整结果（保留4位小数）
print(results_df[column_order].round(4).to_string(index=False))

print("\n" + "="*50)
print(f"最佳模型: {best_model_name}")
print("最佳模型性能指标:")

# 获取最佳模型的所有指标（排除Model列）
best_metrics = results_df[results_df['Model'] == best_model_name][column_order[1:]].iloc[0]
print(best_metrics.to_string())

正在训练 OLS 模型...
OLS 模型训练完成
正在训练 LASSO 模型...
LASSO 模型训练完成
正在训练 Ridge 模型...
Ridge 模型训练完成
正在训练 ElasticNet 模型...
ElasticNet 模型训练完成

各模型性能比较（数值越小越好）:
     Model  In-sample MAE  Out-of-sample MAE  Cross-validation MAE  In-sample RMSE  Out-of-sample RMSE  Cross-validation RMSE  In-sample Median AE  Out-of-sample Median AE
       OLS         0.2382             0.2423                0.2384          0.3222              0.4514                 0.3466               0.1907                   0.1928
     LASSO         0.4959             0.4953                0.4960          0.6471              0.7915                 0.6544               0.3910                   0.3869
     Ridge         0.2381             0.2422                0.2383          0.3222              0.4512                 0.3466               0.1904                   0.1927
ElasticNet         0.4881             0.4879                0.4885          0.6334              0.7905                 0.6420               0.3923                   0.3

In [49]:
results_df

,Model,In-sample MAE,In-sample RMSE,In-sample Median AE,Out-of-sample MAE,Out-of-sample RMSE,Out-of-sample Median AE,Cross-validation MAE,Cross-validation RMSE
0,OLS,0.2382,0.3222,0.1907,0.2423,0.4514,0.1928,0.2384,0.3466
1,LASSO,0.4959,0.6471,0.3910,0.4953,0.7915,0.3869,0.4960,0.6544
2,Ridge,0.2381,0.3222,0.1904,0.2422,0.4512,0.1927,0.2383,0.3466
3,ElasticNet,0.4881,0.6334,0.3923,0.4879,0.7905,0.3885,0.4885,0.6420


In [50]:
results_df.round(4).to_csv('model_results.csv', index=False)
print("结果已保存为 model_results.csv")

结果已保存为 model_results.csv


In [52]:
# 准备真实测试集数据(确保特征与训练集一致)
X_real_test = test_data[features]

# 添加“房屋年限与产权类型”的交互项到测试集
X_real_test["房屋年限 * 产权分数"] = X_real_test["房屋年限分数"] * X_real_test["产权分数"]

# 使用最佳模型进行预测
test_predictions = best_model.predict(X_real_test)

# 创建结果DataFrame (注意对预测值做指数变换还原)
test_results = pd.DataFrame({
    "ID": test_data["ID"].values,
    "Price": np.exp(test_predictions)  # 如果训练时对目标变量取了log，这里需要exp还原
})

# 保存预测结果
test_results.to_csv("submission.csv", index=False)
print("预测结果已保存为 submission.csv")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


预测结果已保存为 submission.csv


In [53]:
# 检查预测值的分布是否合理
plt.figure(figsize=(10,5))
plt.subplot(121)
test_results["Price"].hist(bins=50)
plt.title("预测价格分布")
plt.subplot(122)
train_data["价格"].hist(bins=50)
plt.title("训练集真实价格分布")
plt.show()

<Figure size 720x360 with 2 Axes>